In [1]:
%matplotlib inline

In [2]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from astropy.io import ascii, fits
import matplotlib.pyplot as plt

from itertools import combinations_with_replacement

from dr25 import dr25

/Users/dforeman/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Start by defining some useful quantities:

In [3]:
session = tf.Session()
durations = np.array([1.5, 2.0, 2.5, 3.0, 3.5, 4.5, 5.0, 6.0, 7.5, 9.0, 10.5, 12.0, 12.5, 15.0])
cdpp_cols = ["rrmscdpp01p5", "rrmscdpp02p0", "rrmscdpp02p5", "rrmscdpp03p0",
             "rrmscdpp03p5", "rrmscdpp04p5", "rrmscdpp05p0", "rrmscdpp06p0",
             "rrmscdpp07p5", "rrmscdpp09p0", "rrmscdpp10p5", "rrmscdpp12p0",
             "rrmscdpp12p5", "rrmscdpp15p0"]

Load the stellar properties table and select a subset.

In [24]:
berger = pd.read_csv("data/DR2PapTable1.txt", sep="&")
stlr = ascii.read("data/q1_q17_dr25_stellar.txt").to_pandas()
stlr = pd.merge(berger, stlr, left_on="KIC", right_on="kepid", suffixes=("", "_stlr"))

# Only select dwarfs
m = (4200 <= stlr.teff) & (stlr.teff <= 6100)
m &= stlr.rad <= 1.15
m &= stlr["class\\\\"] == "0\\\\"

# Only include stars with sufficient data coverage.
m &= stlr.dataspan > 365.25*2.
m &= stlr.dutycycle > 0.6
m &= stlr.rrmscdpp07p5 <= 1000.

# Only select stars with logg estimates.
m &= np.isfinite(stlr.logg)

stlr = pd.DataFrame(stlr[m])
print("selected {0} stars".format(len(stlr)))

selected 54837 stars


Load the pixel level injections and Robovetter classifications

In [55]:
# Load the injections
inj = ascii.read("data/kplr_dr25_inj1_plti.txt").to_pandas()
inj["kepid"] = inj.KIC_ID.astype(int)
m = inj.i_depth > 0.0
m &= inj.i_period < 100
m &= inj.EB_injection < 1.0
m &= inj.Offset_from_source < 1.0
inj = inj[m]

# Load the robovetter results
robo = ascii.read("data/kplr_dr25_inj1_tces.txt").to_pandas()

# Only select the injections in our subsample
inj = pd.merge(inj, stlr, on="kepid", suffixes=("", "_stlr"))
inj = pd.merge(inj, robo, how="left", suffixes=("", "_robo"), on="TCE_ID")
m1 = inj.Recovered > 0
m2 = inj.Disp == "PC"
robo_eff = m2.sum() / m1.sum()

print("selected {0} pixel-level injections".format(len(inj)))
print("average robovetter efficiency: {0:.1f}%".format(100*robo_eff))

selected 8170 pixel-level injections
average robovetter efficiency: 93.8%


Find the best fit quadratic limb darkening approximation for each injection

In [47]:
c = np.array(inj[["limbdark_coeff1", "limbdark_coeff2", "limbdark_coeff3", "limbdark_coeff4"]], dtype=float)
b = np.array(inj.i_b)
ror = np.array(inj.i_ror)
r = np.clip(b, 1e-5, 1-1e-5)
mu = np.sqrt(1.0 - r**2)
u = np.array([1.0 - mu, (1.0 - mu)**2])
w = np.array([1.0 - np.sqrt(mu), 1.0 - mu, 1.0 - mu**1.5, 1.0 - mu**2]).T
f = np.sum(c * w, axis=1) / np.sum(u**2, axis=0)
g1, g2 = f[None, :] * u

Build the completeness model in TF

In [48]:
T = tf.float64

R_star = tf.placeholder(T, (None,), name="R_star")
logg_star = tf.placeholder(T, (None,), name="logg_star")
gamma_star = tf.placeholder(T, (None, 2), name="gamma_star")
cdpp_star = tf.placeholder(T, (None, len(durations)), name="cdpp_star")
dataspan_star = tf.placeholder(T, (None,), name="dataspan_star")
dutycycle_star = tf.placeholder(T, (None,), name="dutycycle_star")
R_pl = tf.placeholder(T, (None,), name="R_pl")
P_pl = tf.placeholder(T, (None,), name="P_pl")
b_pl = tf.placeholder(T, (None,), name="b_pl")

# Completeness model
comp_pars = tf.Variable([0.7, 10.0, -0.1], dtype=T, name="comp_pars")
recovered = tf.placeholder(T, (None,), name="recovered")

# Estimate the transit properties
ror_pl = R_pl / R_star
M_star = 10.0**(logg_star-4.437) * tf.square(R_star)
a_pl = 215.0 * M_star**(1.0/3.0) * (P_pl/365.25)**(2.0/3.0)
a_pl_sin_i = tf.sqrt(tf.square(a_pl) - tf.square(R_star * b_pl))
tau_pl = P_pl * tf.asin(R_star * tf.sqrt((1.0+ror_pl)**2-b_pl**2) / a_pl_sin_i) / np.pi

tau_pl = (P_pl*24.0) / 4.0 * R_star / a_pl
depth = (1.0 - dr25.quad(gamma_star[:, 0], gamma_star[:, 1], ror_pl, b_pl)) * 1e6

# Interpolate the CDPP grid
cdpp = dr25.interp(tau_pl, durations, cdpp_star)

# Estimate the MES
ntran = dataspan_star * dutycycle_star / P_pl
mes = tf.sqrt(ntran) * depth / cdpp

# Completeness as a function of MES
pdet = comp_pars[0] / (1.0 + tf.exp(-(mes - comp_pars[1]) * tf.exp(-comp_pars[2])))
pdet = tf.clip_by_value(pdet, 0.0, 1.0)

# Fit completeness parameters to injections
chi2 = tf.norm(tf.where(mes < 15.0, pdet - recovered, tf.zeros_like(pdet)))
opt = tf.contrib.opt.ScipyOptimizerInterface(chi2, [comp_pars])

session.run(tf.global_variables_initializer())

feed = {
    R_star: np.array(inj.rad),
    logg_star: np.array(inj.logg),
    gamma_star: np.vstack((g1, g2)).T,
    cdpp_star: np.array(inj[cdpp_cols]),
    dataspan_star: np.array(inj.dataspan),
    dutycycle_star: np.array(inj.dutycycle),
    R_pl: np.array(inj.i_ror * inj.rad),
    P_pl: np.array(inj.i_period),
    b_pl: np.array(inj.i_b),
    recovered: np.array(((inj.Recovered > 0) & (inj.Disp == "PC")), dtype=float),
}

print("Calibrating completeness model...")
print(session.run(chi2, feed_dict=feed))
opt.minimize(session, feed_dict=feed)
print(session.run(chi2, feed_dict=feed))

Calibrating completeness model...
29.841978011538767
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 28.642368
  Number of iterations: 13
  Number of functions evaluations: 16
28.642367652113215
